In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
df = pd.read_csv('../datasets/movie_data.csv', encoding='utf-8')
df

2024-08-09 01:58:12.568738: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-09 01:58:12.569197: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-09 01:58:12.571049: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-09 01:58:12.576565: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-09 01:58:12.585937: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0
...,...,...
49995,"OK, lets start with the best. the building. al...",0
49996,The British 'heritage film' industry is out of...,0
49997,I don't even know where to begin on this one. ...,0
49998,Richard Tyler is a little boy who is scared of...,0


**전처리 단계**

1. 데이테셋 객체만들고 이를 훈련, 테스트, 검증 데이터셋으로 나눈다.
2. 훈련 데이터셋에 있는 고유한 단어를 찾는다.
3. 고유한 단어를 고유한 저우로 매핑하고 리뷰 텍스트를 정수 배열로 인코딩한다.
4. 모델에 입력하기 위해 데이터셋을 미니 배치로 나눈다.

In [ ]:
# 데이터셋 객체 생성
target = df.pop('sentiment')
ds_raw = tf.data.Dataset.from_tensor_slices((df.values, target.values))

# 데이터셋 확인
for ex in ds_raw.take(3):
    tf.print(ex[0].numpy()[0][:50], ex[1])

b'In 1974, the teenager Martha Moxley (Maggie Grace)' 1
b'OK... so... I really like Kris Kristofferson and h' 0
b'***SPOILER*** Do not read this, if you think about' 0


I0000 00:00:1723136293.936033  273012 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-09 01:58:13.936258: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-08-09 01:58:13.993331: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [ ]:
# 샘플의 개수는 5만 개의 샘플이다.
target.shape

(50000,)

*데이터셋 분리*
(총 5만개)
- 25000개 : 홀드아웃 테스트 데이터셋
- 20000개 : 훈련용 테스트 데이터셋
- 5000개 : 샘플 검증용

In [ ]:
tf.random.set_seed(1)
ds_raw = ds_raw.shuffle(target.shape[0], reshuffle_each_iteration=False)
ds_raw_test = ds_raw.take(25000)
ds_raw_train = ds_raw.take(20000)
ds_raw_valid = ds_raw.take(5000)

*데이터셋 인코딩*
* 훈련 데이터셋에서 고유한 단어를 찾는다 -> 숫자로 문자열을 인코딩해야한다.
* 파이썬 표준 라이브러리에 있는 collections패키지의 Counter 클래스를 사용해한다.


In [5]:
from collections import Counter
# 고유 토큰을 수집하는 코드이다.
tokenizer = tfds.deprecated.text.Tokenizer()
token_counts = Counter()
for ex in ds_raw_train:
    tokens = tokenizer.tokenize(ex[0].numpy()[0])
    token_counts.update(tokens)

print('어휘 사전 크기', len(token_counts))

어휘 사전 크기 87519


2024-08-09 01:58:16.624796: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [6]:
# 고유 토큰을 정수로 인코딩하기
encoder = tfds.deprecated.text.TokenTextEncoder(token_counts)
example_str = 'this is an example!'
print(encoder.encode(example_str))

[95, 10, 170, 4070]


In [7]:
# 텍스트 변환 인코딩 함수 정의 (텍스트_텐서)
def encode(text_tensor, label):
    text = text_tensor.numpy()[0]
    encoded_text = encoder.encode(text)
    return encoded_text, label

In [8]:
# 함수를 TF연산으로 변환하기
def encode_map_fn(text, label):
    return tf.py_function(encode, inp=[text, label], Tout=(tf.int64, tf.int64))


In [9]:
# 텐서플로 연산을 통해서 데이터셋을 정수리스트로 인코딩했다.
ds_train = ds_raw_train.map(encode_map_fn)
ds_valid = ds_raw_valid.map(encode_map_fn)
ds_test = ds_raw_test.map(encode_map_fn)

# ds_train 에서 인코딩된 정수시퀀스 길이 예시 5개 출력
tf.random.set_seed(1)
for ex in ds_train.shuffle(1000).take(5):
    print('sequence length', ex[0].shape)

sequence length (161,)
sequence length (327,)
sequence length (137,)
sequence length (50,)
sequence length (34,)


**문자열 들의 인코딩된 정수 시퀀스 길이가 다르다! (마치, 샘플간의 특성의 개수가 다른상황과 유사하다.)**
- padded_batch()메서드를 이용해서 하나의 배치에 포함되는 모든 원소를 자동으로 0으로 패딩해서 모든 시퀀스가 동일한 길이가 되도록 한다!


In [10]:
# 적용 예시
ds_subset = ds_train.take(8)
for ex in ds_subset:
    print('sample size', ex[0].shape)

ds_batched = ds_subset.padded_batch(4, padded_shapes=([-1], []))
for batch in ds_batched:
    print('to batch case size', batch[0].shape)

#이는 첫번째 4개짜리 배치에서는 226의 크기가 젤 크니깐 226 크기로 0으로 패딩한 결과.
#두번째 4개짜리 배치에서는 413이 가장크니깐 413 크기로 0으로 패딩한 결과.

sample size (114,)
sample size (226,)
sample size (162,)
sample size (83,)
sample size (171,)
sample size (61,)
sample size (308,)
sample size (413,)
to batch case size (4, 226)
to batch case size (4, 413)


2024-08-09 01:58:17.082477: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [11]:
# 실제 데이터셋의 배치 크기를 32의 미니 배치로 나눈다.
train_data = ds_train.padded_batch(32, padded_shapes=([-1], []))
vaild_data = ds_valid.padded_batch(32, padded_shapes=([-1], []))
test_data = ds_test.padded_batch(32, padded_shapes=([-1], []))

## 차원의 저주, 특성 희소성 문제 ##
* 고유한 단어의 수는 수만, 수십만 규모가 될수있다. -> 수십만 차원의 특성을 훈련시켜야하는 경우가 생긴다.
* 또 하나를 제외하고 모든 원소가 0임으로 특성 벡터가 매우 희소해진다.

## 임베딩으로 문제해결 ##
** 임베딩이랑? **
* 임베딩은 원-핫 인코딩과 달리 고정된 길이의 벡터를 사용하여 무한히 많은 실수 표현 (예를들어 [-1. 1])
- *장점*
- 특성 공간 차원축소 -> 차원의 저주 해소
- 임베딩층의 최적화로 중요한 특성 추출이 원핫 인코딩보다 용이하다.

In [12]:
from tensorflow.keras.layers import Embedding
# 임베딩 모델 생성 예시.
model = tf.keras.Sequential()           # 이 임베딩 행렬의 크기는 100 x 6 이다.
model.add(Embedding(input_dim=100,      # 모델이 처리할 단어의 개수, 어휘의 크기
                    output_dim = 6,     # 임베딩 행렬의 차원수
                    input_length=20,
                    name='embed-layer'))
model.build(input_shape=(None, 20))
model.summary() 

/home/jys0714/anaconda3/envs/myenv/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embed-layer (Embedding)         │ (None, 20, 6)          │           600 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 600 (2.34 KB)

 Trainable params: 600 (2.34 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dense

embedding_dim = 20
vocab_size = len(token_counts) + 2 # 아무 단어도 없는 상황에 대한 예약어 0, 현재는 존재하지 않는 단어를 위한 n+1 번째 인덱스 -> 토큰수:n개, 어휘 수:n+2개
tf.random.set_seed(1)

# 모델 생성
bi_lstm_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, name='embed-layer'),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, name='lstm-layer')),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
bi_lstm_model.build(input_shape=(None, vocab_size))
bi_lstm_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embed-layer (Embedding)         │ (None, 87521, 20)      │     1,750,420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 128)            │        43,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,802,261 (6.88 MB)

 Trainable params: 1,802,261 (6.88 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# 컴파일과 훈련
bi_lstm_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                      loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                      metrics=['accuracy'])

history = bi_lstm_model.fit(train_data, validation_data=vaild_data, epochs=10)

test_results = bi_lstm_model.evaluate(test_data)
print('test accuracy : {:.2f}%'. format(test_results[1]*100))

Epoch 1/10
 82/625 ━━━━━━━━━━━━━━━━━━━━ 1:19 147ms/step - accuracy: 0.4862 - loss: 0.6927

KeyboardInterrupt: 